# Homework2

### Mandatory Task:
You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions.

In [ ]:
from pyspark import SparkContext, SparkConf
import findspark
from itertools import combinations

In [2]:
# initializing Spark
findspark.init()
conf = SparkConf().setAppName("FreqItemSets").setMaster("local[*]")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/12 19:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
transactions = sc.textFile("datasets/transaction_dataset.txt").map(lambda line: line.strip().split(" "))
print(transactions.first())

['25', '52', '164', '240', '274', '328', '368', '448', '538', '561', '630', '687', '730', '775', '825', '834']


In [33]:
# Parameters:
s = 0.022 # support threshold - 0.018 there is 3-itemsets as well, but running time is too much for me
num_of_transactions = transactions.count()
frequency_threshold = s * num_of_transactions # how many times should the itemset apper to be frequent
c = 0.6
print(frequency_threshold)

333.036


In [17]:
def generate_candidates(prev_freq_itemsets, freq_1_itemsets, k):
    """Generate candidate k-itemsets by pairing (k-1)-itemsets with 1-itemsets."""
    candidates = set()
    
    for itemset in prev_freq_itemsets:
        for item in freq_1_itemsets:
            # Create a new candidate by adding the 1-itemset to the (k-1)-itemset
            candidate = itemset | frozenset([item])
            
            # Only add if the resulting candidate has exactly k items
            if len(candidate) == k:
                candidates.add(candidate)
    
    return candidates


In [18]:
def prune_candidates(candidates, prev_freq_itemsets):
    """Prune candidate k-itemsets by removing those with infrequent (k-1)-itemset subsets."""
    pruned_candidates = set()
    
    for candidate in candidates:
        is_valid = True
        # Generate all possible (k-1)-itemsets by removing one item at a time
        for item in candidate:
            subset = candidate - frozenset([item])
            # Check if the subset is in the frequent (k-1)-itemsets
            if subset not in prev_freq_itemsets:
                is_valid = False
                break
        # If all (k-1)-subsets are frequent, add candidate to the pruned set
        if is_valid:
            pruned_candidates.add(candidate)
    
    return pruned_candidates


In [19]:
freq_itemsets = dict()
s_count = dict()

item_appear = transactions.flatMap(lambda items: [(item, 1) for item in items]).reduceByKey(lambda x, y: x+y)
freq_1_itemsets_tuples = item_appear.filter(lambda item: item[1] >= frequency_threshold).map(lambda item: (frozenset([item[0]]), item[1])).collect()

freq_1_itemsets = set()

for itemset, count in freq_1_itemsets_tuples:
    freq_itemsets.setdefault(1, set()).add(itemset)
    s_count[itemset] = count
    freq_1_itemsets.update(itemset)

print(len(freq_1_itemsets))
print(s_count)

130
{frozenset({'274'}): 395, frozenset({'538'}): 601, frozenset({'825'}): 472, frozenset({'581'}): 459, frozenset({'674'}): 365, frozenset({'854'}): 429, frozenset({'283'}): 660, frozenset({'883'}): 710, frozenset({'966'}): 586, frozenset({'682'}): 676, frozenset({'782'}): 426, frozenset({'947'}): 570, frozenset({'390'}): 386, frozenset({'279'}): 453, frozenset({'675'}): 441, frozenset({'720'}): 591, frozenset({'914'}): 598, frozenset({'217'}): 860, frozenset({'161'}): 353, frozenset({'175'}): 419, frozenset({'571'}): 441, frozenset({'960'}): 378, frozenset({'862'}): 536, frozenset({'579'}): 343, frozenset({'778'}): 394, frozenset({'70'}): 361, frozenset({'204'}): 345, frozenset({'419'}): 758, frozenset({'722'}): 884, frozenset({'526'}): 418, frozenset({'541'}): 550, frozenset({'631'}): 390, frozenset({'780'}): 347, frozenset({'956'}): 557, frozenset({'145'}): 735, frozenset({'676'}): 449, frozenset({'885'}): 473, frozenset({'522'}): 408, frozenset({'12'}): 530, frozenset({'296'}): 36

In [23]:
print(f"freq_1_itemsets: {freq_1_itemsets}")

# Convert individual items in freq_1_itemsets to frozensets
freq_1_itemsets = set(frozenset([item]) for item in freq_1_itemsets)

freq_1_itemsets: {'947', '548', '8', '112', '529', '914', '752', '676', '854', '956', '598', '57', '569', '998', '510', '93', '346', '634', '803', '862', '239', '523', '883', '120', '27', '758', '631', '285', '793', '438', '722', '614', '829', '579', '937', '145', '675', '283', '161', '541', '175', '538', '413', '778', '561', '72', '217', '526', '788', '888', '12', '480', '653', '921', '782', '151', '487', '489', '795', '236', '798', '885', '694', '918', '21', '919', '620', '419', '69', '204', '522', '886', '350', '140', '684', '362', '274', '54', '470', '692', '581', '296', '39', '775', '75', '368', '797', '944', '825', '132', '789', '661', '48', '477', '71', '617', '509', '606', '392', '78', '593', '780', '571', '381', '895', '471', '70', '874', '177', '390', '279', '766', '460', '494', '205', '183', '844', '276', '354', '960', '720', '597', '242', '682', '32', '871', '674', '401', '38', '966'}


In [21]:
#freq_k_itemsets = set(frozenset([item]) for item in freq_1_itemsets)
#freq_itemsets[1] = freq_k_itemsets

print(f"freq_itemsets[k-1]: {freq_itemsets[k-1]}")
print(f"freq_1_itemsets: {freq_1_itemsets}")


freq_itemsets[k-1]: {frozenset({'921'}), frozenset({'279'}), frozenset({'276'}), frozenset({'775'}), frozenset({'183'}), frozenset({'598'}), frozenset({'177'}), frozenset({'653'}), frozenset({'78'}), frozenset({'120'}), frozenset({'350'}), frozenset({'460'}), frozenset({'682'}), frozenset({'413'}), frozenset({'151'}), frozenset({'541'}), frozenset({'883'}), frozenset({'914'}), frozenset({'140'}), frozenset({'788'}), frozenset({'419'}), frozenset({'825'}), frozenset({'829'}), frozenset({'579'}), frozenset({'895'}), frozenset({'692'}), frozenset({'438'}), frozenset({'38'}), frozenset({'780'}), frozenset({'12'}), frozenset({'27'}), frozenset({'797'}), frozenset({'8'}), frozenset({'844'}), frozenset({'529'}), frozenset({'477'}), frozenset({'48'}), frozenset({'722'}), frozenset({'72'}), frozenset({'480'}), frozenset({'21'}), frozenset({'69'}), frozenset({'471'}), frozenset({'597'}), frozenset({'32'}), frozenset({'217'}), frozenset({'522'}), frozenset({'354'}), frozenset({'93'}), frozenset({

In [24]:
k=2

while True:
    candidates_k = generate_candidates(freq_itemsets[k-1], freq_1_itemsets, k)
    if not candidates_k:
        break
    
    #Pruning:
    candidates_k = prune_candidates(candidates_k, freq_itemsets[k-1])
    if not candidates_k:
        break
    
    candidates_k_rdd = sc.parallelize(list(candidates_k)).map(lambda item: (item, 1)) # every element is in (itemset, 1) format
    
    transaction_k_itemsets = transactions.flatMap(lambda transaction: [frozenset(combo) for combo in combinations(transaction, k)]).map(lambda x: (x, 1)) # contains the subsets as well from candidates_k_rdd in the format (itemset, 1)
        
    # Join candidate itemsets with transaction itemsets to count support
    candidate_counts = candidates_k_rdd.join(transaction_k_itemsets).map(lambda x: (x[0], x[1][0] + x[1][1])).reduceByKey(lambda a, b: a + b)
    
    freq_k_itemsets = candidate_counts.filter(lambda x: x[1] >= frequency_threshold).map(lambda x: x[0]).collect()
        
    if not freq_k_itemsets:
        break

    freq_itemsets[k] = set()
    for itemset, count in freq_k_itemsets:
        freq_itemsets[k].add(itemset)
        s_count[itemset] = count
    k += 1

In [25]:
print(freq_itemsets)

{1: {frozenset({'921'}), frozenset({'279'}), frozenset({'276'}), frozenset({'775'}), frozenset({'183'}), frozenset({'598'}), frozenset({'177'}), frozenset({'653'}), frozenset({'78'}), frozenset({'120'}), frozenset({'350'}), frozenset({'460'}), frozenset({'682'}), frozenset({'413'}), frozenset({'151'}), frozenset({'541'}), frozenset({'883'}), frozenset({'914'}), frozenset({'140'}), frozenset({'788'}), frozenset({'419'}), frozenset({'825'}), frozenset({'829'}), frozenset({'579'}), frozenset({'895'}), frozenset({'692'}), frozenset({'438'}), frozenset({'38'}), frozenset({'780'}), frozenset({'12'}), frozenset({'27'}), frozenset({'797'}), frozenset({'8'}), frozenset({'844'}), frozenset({'529'}), frozenset({'477'}), frozenset({'48'}), frozenset({'722'}), frozenset({'72'}), frozenset({'480'}), frozenset({'21'}), frozenset({'69'}), frozenset({'471'}), frozenset({'597'}), frozenset({'32'}), frozenset({'217'}), frozenset({'522'}), frozenset({'354'}), frozenset({'93'}), frozenset({'871'}), frozens

In [29]:
all_frequent_itemsets = []
for size, itemsets in freq_itemsets.items():
    for itemset in itemsets:
        all_frequent_itemsets.append((frozenset(itemset), size, s_count[itemset]))

all_frequent_itemsets = sorted(all_frequent_itemsets, key=lambda x: (x[1], sorted(x[0])))


In [30]:
print("\nAll Frequent Itemsets:")
for itemset, size, count in all_frequent_itemsets:
    print(f"Itemset: {set(itemset)}, Size: {size}, Support Count: {count}")




All Frequent Itemsets:
Itemset: {'112'}, Size: 1, Support Count: 410
Itemset: {'12'}, Size: 1, Support Count: 530
Itemset: {'120'}, Size: 1, Support Count: 730
Itemset: {'132'}, Size: 1, Support Count: 427
Itemset: {'140'}, Size: 1, Support Count: 393
Itemset: {'145'}, Size: 1, Support Count: 735
Itemset: {'151'}, Size: 1, Support Count: 394
Itemset: {'161'}, Size: 1, Support Count: 353
Itemset: {'175'}, Size: 1, Support Count: 419
Itemset: {'177'}, Size: 1, Support Count: 692
Itemset: {'183'}, Size: 1, Support Count: 618
Itemset: {'204'}, Size: 1, Support Count: 345
Itemset: {'205'}, Size: 1, Support Count: 552
Itemset: {'21'}, Size: 1, Support Count: 379
Itemset: {'217'}, Size: 1, Support Count: 860
Itemset: {'236'}, Size: 1, Support Count: 443
Itemset: {'239'}, Size: 1, Support Count: 388
Itemset: {'242'}, Size: 1, Support Count: 347
Itemset: {'27'}, Size: 1, Support Count: 348
Itemset: {'274'}, Size: 1, Support Count: 395
Itemset: {'276'}, Size: 1, Support Count: 410
Itemset: {'27

### Association Rule Generation

In [34]:
association_rules = []

for itemset, size, count in all_frequent_itemsets:
    if size < 2:
        continue
    # Generate all non-empty proper subsets of the itemset
    subsets = []
    for r in range(1, size):
        subsets.extend(combinations(itemset, r))
    
    # Process each subset to form rules
    for subset in subsets:
        antecedent = frozenset(subset)
        consequent = itemset - antecedent
        if not consequent:
            continue
        # Retrieve support counts
        support_itemset = int(count)
        support_antecedent = int(s_count.get(antecedent, 0))
        if support_antecedent == 0:
            continue  # Avoid division by zero
        confidence = support_itemset / support_antecedent
        if confidence >= c:
            association_rules.append((set(antecedent), set(consequent), support_itemset, support_antecedent, confidence))



In [35]:
association_rules = sorted(association_rules, key=lambda x: x[4], reverse=True)

for antecedent, consequent, support_L, support_A, confidence in association_rules:
    print(f"Rule: {antecedent} -> {consequent}, Support(L): {support_L}, Support(A): {support_A}, Confidence: {confidence:.2f}")


Rule: {'8'} -> {'2', '6'}, Support(L): 368, Support(A): 431, Confidence: 0.85
Rule: {'8'} -> {'2', '9'}, Support(L): 368, Support(A): 431, Confidence: 0.85
